# synthetic time series for evaluation

In [13]:
import matplotlib.pyplot as plt
# import timesynth as ts
import numpy as np
import pandas as pd
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [14]:
# channel number
channel_num = 4
# segment number
seg_num = 20
seg_len = 1000
# must be lower than seg_num.
state_num = 5
ts_num = 100

In [15]:
# Generate segment json
def generate_seg_json(seg_num, state_num, random_state=None):
    # Config seed to generate determinally.
    if random_state is not None:
        np.random.seed(random_state)
    seg_json = {}
    state_list = np.random.randint(state_num, size=seg_num)
    total_len = 1000
    for i, state in zip(range(seg_num), state_list):
        seg_len = 1000
        seg_json[total_len]=state
        total_len += seg_len
    return seg_json

In [16]:
from shape import RMDF

shape_len = 200
num_shape = 5
def gen_channel_from_json(seg_json):
    state_list = [seg_json[seg] for seg in seg_json]
    true_state_num = len(set(state_list))
    # This is an object list.
    rmdf_list = [RMDF.RMDF(depth=5) for i in range(true_state_num)]
    for rmdf in rmdf_list:
        rmdf.gen_anchor()
    seg_list = []
    shape_len = np.random.randint(100,400)
    for state in state_list:
        seg = [rmdf_list[state].gen(forking_depth=1, length=shape_len) for i in range(num_shape)]
        seg_list.append(np.concatenate(seg))
    result = np.concatenate(seg_list)
    return result

def gen_from_json(seg_json):
    # generate channel respectively.
    channel_list = [gen_channel_from_json(seg_json) for i in range(channel_num)]
    return np.stack(channel_list).T

In [17]:
seg_json_list = []
for i in range(ts_num):
    seg_json =generate_seg_json(seg_num, state_num)
    while len(set([seg_json[key] for key in list(seg_json)])) != state_num:
        seg_json = generate_seg_json(seg_num, state_num)
    seg_json_list.append(seg_json)

In [18]:
from TSpy.label import seg_to_label, adjust_label
import os

path = '../data/synthetic_data_for_segmentation2'
for i, seg_json in enumerate(seg_json_list):
    data = gen_from_json(seg_json)
    # plot_mulvariate_time_series(data)
    label = adjust_label(seg_to_label(seg_json))
    df = pd.DataFrame(data)
    df.insert(4, 'label', label)
    if not os.path.exists(path):
        os.mkdir(path)
    df.to_csv(path+'/test'+str(i)+'.csv', header=False, index=False)

# path = '../data/synthetic_data_for_Autoplait'
# if not os.path.exists(path):
#     os.mkdir(path)
# with open(path+'/list', 'w') as f:
#     for i in range(ts_num):
#         f.writelines('../data/synthetic_data_for_Autoplait/test'+str(i)+'.csv\n')
# for i, seg_json in enumerate(seg_json_list):
#     data = gen_from_json(seg_json)
#     # plot_mulvariate_time_series(data)
#     label = adjust_label(seg_to_label(seg_json))
#     df = pd.DataFrame(data).round(4)
#     df.insert(4, 'label', label)
#     if not os.path.exists(path):
#         os.mkdir(path)
#     df.to_csv(path+'/test'+str(i)+'.csv', header=False, index=False, sep=' ')

ValueError: all input arrays must have the same shape